In [ ]:
import argparse
from datetime import datetime
import os
from pathlib import Path
import torch
import torch.optim as optim
import optuna
from optuna.trial import TrialState

from torch.nn import MSELoss, L1Loss
from processing.solver import Solver
import utils.utils_args as ut
import preprocessing.preprocessing as prep
from processing.networks.unetVariants import UNetNoPad2
from main import read_cla
from preprocessing.data_init import init_data, load_all_datasets_in_full
from postprocessing.visualization import visualizations
from postprocessing.measurements import measure_losses_paper24

optuna:
-x num_convs (1,2,3,4)
-x complexity (=init_features) (2,64) (log)
-x kernel_size (3,5,7)
-x Adam weight decay (0, 1e-5)
-x lr (1e-3, 1e-4, 1e-5)
-x alpha: (0, 1, 0.5, 0.25, 0.75)
    loss funciton: (1-alpha)*MSE+alpha*MAE
-x batch size

overall loss for comparison: MSE

In [ ]:
def objective(trial):

    # Get the dataset.
    args["inputs"] = "pki"
    input_channels, output_channels, dataloaders = init_data(args)

    # Generate the model.
    depth = trial.suggest_categorical("depth", [2, 3, 4, 5])
    init_features = trial.suggest_categorical("init_features", [16, 32, 64])
    kernel_size = trial.suggest_int("kernel_size", 3, 4, 5)

    model = None

    # Generate the optimizers.
    lr = trial.suggest_categorical("lr", [1e-3, 1e-4, 1e-5])
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam"]) #, "RMSprop", "SGD"])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Training of the model.
    solver = Solver(model, dataloaders["train"], dataloaders["val"], loss_func=L1Loss(), opt=optimizer, learning_rate=lr)
    try:
        loss = solver.train(trial, args)
        model.save(args["destination"], model_name = f"model_trial_{trial.number}.pt")
        solver.save_metrics(args["destination"], model.num_of_params(), args["epochs"], training_time, args["device"])

    except Exception as e:
        print(f"Training failed with exception: {e}")
        loss = 1

    return loss

# main

In [ ]:
OUTPUT_DIR = Path("/scratch/e451412/outputs")
NAME_DIR = None
args["destination"] = OUTPUT_DIR / NAME_DIR

study = optuna.create_study(direction="minimize", storage=f"sqlite:///{OUTPUT_DIR.as_posix()}/{args['destination'].name}/trials.db", study_name="simplified_cnn", load_if_exists=True)
study.optimize(objective, n_trials=50)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))

print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print("    {}: {}".format(key, value))